In [ ]:
import os
import importlib

%cd /mnt/home/acanatar/jupyter/SNAP

data_root = './data/'
os.makedirs(data_root, exist_ok=True)

device = 'cuda'

In [ ]:
import dimensionality
importlib.reload(dimensionality)
importlib.reload(dimensionality.models)
importlib.reload(dimensionality.wrapper)
importlib.reload(dimensionality.metrics)
importlib.reload(dimensionality.experiment)
importlib.reload(dimensionality.brainscore_data)
importlib.reload(dimensionality.regression_utils)
importlib.reload(dimensionality.utils)

from dimensionality.wrapper import TorchWrapper

from dimensionality.experiment import Experiment
from dimensionality.brainscore_data import get_neural_data

import dimensionality.models as models
from dimensionality.regression_utils import regression_metric

In [ ]:
region='V1'
dataset = 'dicarlo'
image_type = 'original'  # Only matters for Cadena dataset

data_loader_neural, images, labels = get_neural_data(region=region, dataset=dataset, image_type=image_type)

# Brainscore Experiments

In [35]:
dataset = 'dicarlo'
image_type = 'original'  # Only matters for Cadena dataset

regionNames = ['V1',
               'V2',
               'V4',
               'IT'
               ]

activation_pooling = [None,
                      #   'MaxPool_(1,1)',
                      #   'AvgPool_(1,1)',
                      ]

rand_proj_dim = None
pretrained = {True: 'pretrained',
              False: 'untrained'
              }


modelNames = ['alexnet', 'resnet18', 'resnet50',
              # 'robust_resnet50_l2_3', 'moco_resnet50',
              'barlowtwins', 'robust_resnet50_linf_4'
              ]


for region in regionNames:

    data_loader_neural, images, labels = get_neural_data(region=region, dataset=dataset, image_type=image_type)

    for model_name in modelNames:

        for pooling in activation_pooling:
            for trained in [True, False]:

                data_dir = os.path.join(data_root, f"data_{pooling}_RandProj_{rand_proj_dim}")
                data_fname = os.path.join(data_dir, f"{region}_data_{model_name}_{pretrained[trained]}.npz")
                os.makedirs(data_dir, exist_ok=True)
                print(data_fname)

                # Get the model
                model_kwargs = {'name': model_name,
                                'pretrained': trained,
                                'device': device}
                model, layers, identifier = models.get_model(**model_kwargs)
                model_wrapped = TorchWrapper(model, layers=layers, identifier=identifier, activation_pooling=pooling)

                # Create the Experiment Class and pass additional metrics
                regression_kwargs = {'num_trials': 5,
                                     'reg': 1e-14,
                                     'num_points': 5,
                                     }

                metric_fns = [regression_metric]
                exp = Experiment(model_wrapped, metric_fns=metric_fns, rand_proj_dim=rand_proj_dim)

                # Extract the activations of the layers passed above using data_loader (only uses the inputs)
                exp.get_activations(data_loader_neural())

                # Compute metrics
                metric_kwargs = {'debug': False,
                                 'threshold': 1-1e-8,
                                 'epsilon': 1e-14
                                 } | regression_kwargs

                exp_metrics = exp.compute_metrics(images=images, labels=labels, **metric_kwargs)
                layers = exp_metrics['layers']

                np.savez(data_fname, exp_metrics=exp_metrics, layers=layers, metric_kwargs=metric_kwargs)

Layer:   6%|▌         | 1/18 [00:17<04:56, 17.43s/it]


KeyboardInterrupt: 